In [27]:
import numpy as np
import pandas as pd
import os
import plotly.express as px
from tensorflow.keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('../data/all_stock.csv')

In [3]:
df['date_only'] = df['Datetime'].apply(lambda x : x[:10])
df = df[df['date_only'] == '2024-08-12']
df = df[['Datetime','Open']]

In [4]:
total_shift = 30
col_list = ['Open']
for i in range(1,total_shift+1):
    col_list.append('shift_' + str(i))

processed_df = df[['Open']].copy()
df = df[total_shift:]

In [5]:
scaler = MinMaxScaler()
processed_df['Open'] = scaler.fit_transform(processed_df)
for i in range(1,total_shift+1):
        processed_df['shift_' + str(i)] = processed_df['Open'].shift(i)
processed_df = processed_df.dropna()

In [9]:
model = load_model('../model.h5')

In [10]:
X_test = list(processed_df.drop('Open',axis=1).values)
X_test = [row[::-1] for row in X_test]
X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [15]:
df['preds'] = model.predict(X_test)
df['Open_pred'] = scaler.inverse_transform(df['preds'].values.reshape(-1,1))

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


In [22]:
y_true = df['Open'].values
y_pred = df['Open_pred'].values

In [25]:
print('MAE : ',mean_absolute_error(y_true,y_pred))
print('MSE : ', mean_squared_error(y_true, y_pred))

MAE :  6.396937273550725
MSE :  68.7570479793825


In [28]:
fig = px.line(df, x = 'Datetime', y = ['Open','Open_pred'])
fig.show()